In [1]:
import cv2
import math
import numpy as np
from imutils.video import FileVideoStream
from imutils.video import FPS
import time
import imutils

# Загрузка видео
video_path = r"C:\Users\Roadmarshal\Downloads\Video Example.mp4"
fvs = FileVideoStream(video_path).start()
time.sleep(1.0)  # Даем время для инициализации видеопотока

# Параметры камеры и видео
sensor_width_mm = 5.64
frame_width = 1920
frame_height = 1080
focal_length_mm = 5.1

# Функции расчета угла и дистанции
def calculate_horiz_angle_view(f, sensor_width):
    return 2 * math.atan((sensor_width / (2 * f)))

def object_distance(focal_length, pixel_width, real_width):
    fov = calculate_horiz_angle_view(focal_length, sensor_width_mm)
    focal_length_pixels = (frame_width / 2) / math.tan(fov / 2)
    distance = (real_width * focal_length_pixels) / pixel_width
    return distance

horizontal_angle_view = calculate_horiz_angle_view(focal_length_mm, sensor_width_mm)
fps = FPS().start()

while fvs.more():
    frame = fvs.read()
    if frame is None:
        break
    
    frame = imutils.resize(frame, width=frame_width)
    x, y, w, h = 100, 100, 200, 80
    mig_wing_span = 11.99
    distance = object_distance(focal_length_mm, w, mig_wing_span)

    center_frame = (int(frame_width / 2), int(frame_height / 2))
    center_object = (int(x + w / 2), int(y + h / 2))
    deviation_x = center_object[0] - center_frame[0]
    deviation_y = center_object[1] - center_frame[1]

    cv2.line(frame, center_frame, (center_frame[0], frame_height), (0, 0, 255), 2)
    cv2.line(frame, (0, center_frame[1]), (frame_width, center_frame[1]), (0, 0, 255), 2)
    for r in range(100, 501, 100):
        cv2.circle(frame, center_frame, r, (0, 0, 255), 2)

    text_start_x, text_start_y = 10, 10
    cv2.putText(frame, f"Distance: {distance:.2f} m", (text_start_x, text_start_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"x deviation: {deviation_x:.2f}", (text_start_x, text_start_y + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"y deviation: {deviation_y:.2f}", (text_start_x, text_start_y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    fps.update()

fps.stop()
cv2.destroyAllWindows()
fvs.stop()
